In [60]:
from credentials import my_SteamID, my_APIkey

getList_URL = f"http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={my_APIkey}&steamid={my_SteamID}&relationship=friend"
print(getList_URL)

http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key=3B90B4B6F5209358DD991B763FF56CE9&steamid=76561198048688318&relationship=friend


In [61]:
import urllib
file = urllib.request.urlopen(getList_URL)
content = file.read().decode('utf-8')

import json
my_friendlist = json.loads(content)

In [62]:
import pandas as pd
my_friendlist = pd.DataFrame(my_friendlist["friendslist"]["friends"])
my_friendlist



,steamid,relationship,friend_since
0,76561197960487372,friend,1360424195
1,76561197962583100,friend,1397381161
2,76561197970586960,friend,1612381871
3,76561197977569434,friend,1345493291
4,76561197979479074,friend,1348602855
5,76561197979975796,friend,1353522397
6,76561197981033122,friend,1348424175
7,76561197982396704,friend,1352058037
8,76561197985222289,friend,1376939017
9,76561197989292226,friend,1354725719


In [91]:
def get_friendlist(steamid, APIkey):
    getList_URL = f"http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={APIkey}&steamid={steamid}&relationship=friend"
    file = urllib.request.urlopen(getList_URL)
    content = file.read().decode('utf-8')
    friendlist = json.loads(content)
    friendlist = pd.DataFrame(friendlist["friendslist"]["friends"])
    return friendlist

In [115]:
def get_friend_info(friendlist, APIkey):
    keys=["steamid", "personaname", "personastate"]
    out=[]
    for id in friendlist["steamid"]:
        getFriend_URL = f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={APIkey}&steamids={id}"
        file = urllib.request.urlopen(getFriend_URL)
        content = file.read().decode('utf-8')
        friendstatus = json.loads(content)
        try:
            friendstatus = friendstatus["response"]["players"][0]
            out.append([friendstatus.get(key) for key in keys])
        except:
            continue
    return pd.DataFrame(out, columns=keys)

In [131]:
def get_ingame_friends(friendlist, APIkey):
    keys=["steamid", "personaname", "gameextrainfo"]
    out=[]
    for id in friendlist["steamid"]:
        getFriend_URL = f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={APIkey}&steamids={id}"
        file = urllib.request.urlopen(getFriend_URL)
        content = file.read().decode('utf-8')
        friendstatus = json.loads(content)
        try:
            friendstatus = friendstatus["response"]["players"][0]
            if friendstatus["gameextrainfo"] != None:
                out.append([friendstatus.get(key) for key in keys])
        except:
            continue
    return pd.DataFrame(out, columns=keys)

In [133]:
my_friendlist = get_friendlist(my_SteamID, my_APIkey)
#my_friend_info = get_friend_info(my_friendlist, my_APIkey)
my_ingame_friends = get_ingame_friends(my_friendlist, my_APIkey)
my_ingame_friends

,steamid,personaname,gameextrainfo
0,76561198026715831,[WC] Scott Sterling,Sands of Salzaar
1,76561198059254917,Pail,Warhammer: Vermintide 2


In [116]:
# online friends
my_friend_info[my_friend_info["personastate"] != 0].sort_values("personastate")

,steamid,personaname,personastate
18,76561198007361576,mayorpain11,1
25,76561198026715831,[WC] Scott Sterling,1
27,76561198034930569,Shadowblade,1
48,76561198132034600,faber.alexander,1
54,76561199037315001,[WC] Fleischi,1
4,76561197979479074,LeadIng,3
41,76561198068931354,chrisse286,3


In [124]:
id= "76561198026715831"
getFriend_URL = f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={my_APIkey}&steamids={id}"
file = urllib.request.urlopen(getFriend_URL)
content = file.read().decode('utf-8')
friendstatus = json.loads(content)
friendstatus

{'response': {'players': [{'steamid': '76561198026715831',
    'communityvisibilitystate': 3,
    'profilestate': 1,
    'personaname': '[WC] Scott Sterling',
    'profileurl': 'https://steamcommunity.com/id/socing/',
    'avatar': 'https://avatars.akamai.steamstatic.com/c18e067d5c5b0654369a42ae92a32ac105f42e21.jpg',
    'avatarmedium': 'https://avatars.akamai.steamstatic.com/c18e067d5c5b0654369a42ae92a32ac105f42e21_medium.jpg',
    'avatarfull': 'https://avatars.akamai.steamstatic.com/c18e067d5c5b0654369a42ae92a32ac105f42e21_full.jpg',
    'avatarhash': 'c18e067d5c5b0654369a42ae92a32ac105f42e21',
    'lastlogoff': 1673856389,
    'personastate': 1,
    'realname': 'Scott Sterling',
    'primaryclanid': '103582791463890108',
    'timecreated': 1276860108,
    'personastateflags': 0,
    'gameextrainfo': 'Sands of Salzaar',
    'gameid': '1094520',
    'loccountrycode': 'BD'}]}}

In [126]:
id= "76561198059254917"
getFriend_URL = f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={my_APIkey}&steamids={id}"
file = urllib.request.urlopen(getFriend_URL)
content = file.read().decode('utf-8')
friendstatus = json.loads(content)
friendstatus

{'response': {'players': [{'steamid': '76561198059254917',
    'communityvisibilitystate': 3,
    'profilestate': 1,
    'personaname': 'Pail',
    'commentpermission': 2,
    'profileurl': 'https://steamcommunity.com/id/Pail_1/',
    'avatar': 'https://avatars.akamai.steamstatic.com/eef75511ba1f16b66e9ac45346e7c0f790a6551a.jpg',
    'avatarmedium': 'https://avatars.akamai.steamstatic.com/eef75511ba1f16b66e9ac45346e7c0f790a6551a_medium.jpg',
    'avatarfull': 'https://avatars.akamai.steamstatic.com/eef75511ba1f16b66e9ac45346e7c0f790a6551a_full.jpg',
    'avatarhash': 'eef75511ba1f16b66e9ac45346e7c0f790a6551a',
    'lastlogoff': 1673473650,
    'personastate': 1,
    'realname': 'Schniin',
    'primaryclanid': '103582791429521408',
    'timecreated': 1329986845,
    'personastateflags': 0,
    'loccountrycode': 'DE'}]}}